<a href="https://colab.research.google.com/github/nadiralam/Health_-_Fitness_Planner_Agent/blob/main/health_fitness_agentPlanar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q gradio agno

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 726.7/726.7 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 472.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 18.0 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
api_key = userdata.get('Gemini_key')

In [ ]:
import gradio as gr
from agno.agent import Agent
from agno.models.google import Gemini


In [ ]:
# Global session state
session_state = {
    "dietary_plan": {},
    "fitness_plan": {},
    "qa_pairs": [],
    "plans_generated": False
}


In [ ]:
# Function to generate plans
def generate_plans(name, age, height, weight, sex, activity_level, fitness_goals, dietary_preferences, api_key):
    try:
        gemini_model = Gemini(id="gemini-1.5-flash", api_key=api_key)

        user_profile = f"""
        Name: {name}
        Age: {age}
        Weight: {weight}kg
        Height: {height}cm
        Sex: {sex}
        Activity Level: {activity_level}
        Dietary Preferences: {dietary_preferences}
        Fitness Goals: {fitness_goals}
        """

        dietary_agent = Agent(
            name="Dietary Expert",
            role="Provides personalized dietary recommendations",
            model=gemini_model,
            instructions=[
                "Consider the user's input, including dietary restrictions and preferences.",
                "Suggest a detailed meal plan for the day, including breakfast, lunch, dinner, and snacks.",
                "Provide a brief explanation of why the plan is suited to the user's goals.",
                "Focus on clarity, coherence, and quality of the recommendations.",
            ]
        )

        fitness_agent = Agent(
            name="Fitness Expert",
            role="Provides personalized fitness recommendations",
            model=gemini_model,
            instructions=[
                "Provide exercises tailored to the user's goals.",
                "Include warm-up, main workout, and cool-down exercises.",
                "Explain the benefits of each recommended exercise.",
                "Ensure the plan is actionable and detailed.",
            ]
        )

        dietary_plan_response = dietary_agent.run(user_profile)
        fitness_plan_response = fitness_agent.run(user_profile)

        dietary_plan = {
            "why_this_plan_works": "High Protein, Healthy Fats, Moderate Carbohydrates, and Caloric Balance",
            "meal_plan": dietary_plan_response.content,
            "important_considerations": """
- Hydration: Drink plenty of water throughout the day
- Electrolytes: Monitor sodium, potassium, and magnesium levels
- Fiber: Ensure adequate intake through vegetables and fruits
- Listen to your body: Adjust portion sizes as needed
"""
        }

        fitness_plan = {
            "goals": "Build strength, improve endurance, and maintain overall fitness",
            "routine": fitness_plan_response.content,
            "tips": """
- Track your progress regularly
- Allow proper rest between workouts
- Focus on proper form
- Stay consistent with your routine
"""
        }

        session_state["dietary_plan"] = dietary_plan
        session_state["fitness_plan"] = fitness_plan
        session_state["plans_generated"] = True

        return (
            f"## 🥗 Why this Plan Works\n{dietary_plan['why_this_plan_works']}\n\n"
            f"## 📋 Meal Plan\n{dietary_plan['meal_plan']}\n\n"
            f"### ⚠️ Considerations:\n{dietary_plan['important_considerations']}",
            f"## 🏋️‍♂️ Fitness Goals\n{fitness_plan['goals']}\n\n"
            f"## 💪 Exercise Routine\n{fitness_plan['routine']}\n\n"
            f"### 💡 Tips:\n{fitness_plan['tips']}"
        )

    except Exception as e:
        return f"❌ Error: {e}", ""


In [ ]:
# Function to ask questions
def ask_question(question, api_key):
    if not session_state["plans_generated"]:
        return "Please generate a plan first."

    dietary_plan = session_state["dietary_plan"]
    fitness_plan = session_state["fitness_plan"]

    context = f"Dietary Plan: {dietary_plan.get('meal_plan', '')}\n\nFitness Plan: {fitness_plan.get('routine', '')}"
    full_context = f"{context}\nUser Question: {question}"

    try:
        gemini_model = Gemini(id="gemini-1.5-flash", api_key=api_key)
        agent = Agent(model=gemini_model)
        run_response = agent.run(full_context)
        answer = run_response.content if hasattr(run_response, 'content') else "No answer generated."
        session_state["qa_pairs"].append((question, answer))
        return answer
    except Exception as e:
        return f"❌ Error getting answer: {e}"



In [ ]:
# Gradio UI
with gr.Blocks(title="AI Health & Fitness Planner") as demo:
    gr.Markdown("## 🏋️‍♂️ AI Health & Fitness Planner")
    gr.Markdown("Achieve your goals with smart, personalized diet and fitness plans—crafted just for you by AI!")

    with gr.Tab("🔐 API & Profile Setup"):
        api_key = gr.Textbox(label="Gemini API Key", type="password", placeholder="Paste your API key here")
        name = gr.Textbox(label="Name")
        age = gr.Number(label="Age", value=25, precision=0)
        height = gr.Number(label="Height (cm)", value=170.0)
        weight = gr.Number(label="Weight (kg)", value=65.0)
        sex = gr.Dropdown(label="Sex", choices=["Male", "Female", "Other"])
        activity_level = gr.Dropdown(label="Activity Level", choices=["Sedentary", "Lightly Active", "Moderately Active", "Very Active", "Extremely Active"])
        fitness_goals = gr.Dropdown(label="Fitness Goals", choices=["Lose Weight", "Gain Muscle", "Endurance", "Stay Fit", "Strength Training"])
        dietary_preferences = gr.Dropdown(label="Dietary Preferences", choices=["Vegetarian","Non vegetarian", "Keto", "Gluten Free", "Low Carb", "Dairy Free"])

        generate_button = gr.Button("🎯 Generate Plan")
        dietary_output = gr.Markdown()
        fitness_output = gr.Markdown()

    generate_button.click(
        generate_plans,
        inputs=[name, age, height, weight, sex, activity_level, fitness_goals, dietary_preferences, api_key],
        outputs=[dietary_output, fitness_output]
    )

    with gr.Tab("💬 Ask About Your Plan"):
        question = gr.Textbox(label="Ask a Question")
        answer_output = gr.Markdown()
        ask_button = gr.Button("Get Answer")

        ask_button.click(
            ask_question,
            inputs=[question, api_key],
            outputs=answer_output
        )




In [ ]:
demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://11f4a51b3dbd42bd86.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
